# 4. 단어 추출 word extraction

## 4-a. 단어 추출을 하는 이유
- 커뮤니티 text는 신조어, 은어가 많다.
- 토큰화를 하기 위해선, 신조어와 은어를 추출해야 한다.

## 4-b. 단어 추출 계획
1. text로만 이루어진 txt 파일 만들기
2. soynlp 의 WordExtractor 를 이용하여 단어 추출
3. 단어와 수치 데이터를 csv 파일로 저장

## 4-c. 단어 추출 진행

### 4-c-1. 파일 불러오기

In [145]:
import pandas as pd
df = pd.read_csv("3_spaced_text.csv", encoding="utf-8")
df.head(3)

,search_keyword,date_created,time_created,writer,is_reply,id,spaced_text
0,JYP,2023-05-16,17:52:40,ㅇㅇ(118.34),0,10100730087,야 개보지년들아 JYP 주가 봤냐.. 미쳣다 지금... 개잡주 오르는건 그러려니 하...
1,JYP,2023-05-16,17:52:48,ㅇㅇ(118.34),1,10100730087,ㅠㅠ
2,JYP,2023-05-16,18:04:04,ㅇㅇ(203.227),1,10100730087,ㅠㅠ


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891014 entries, 0 to 891013
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   search_keyword  891014 non-null  object
 1   date_created    891014 non-null  object
 2   time_created    891014 non-null  object
 3   writer          891014 non-null  object
 4   is_reply        891014 non-null  int64 
 5   id              891014 non-null  int64 
 6   spaced_text     891014 non-null  object
dtypes: int64(2), object(5)
memory usage: 47.6+ MB


### 4-c-2. corpus를 list 형태로 만든다

In [146]:
corpus = df['spaced_text'].tolist()

### 4-c-3. word_extractor 학습시키기

In [147]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(min_frequency=100,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)
word_extractor.train(corpus) # list of str or like
words = word_extractor.extract()

training was done. used memory 2.740 Gbse memory 3.282 Gb
all cohesion probabilities was computed. # words = 27749
all branching entropies was computed # words = 1146192
all accessor variety was computed # words = 1146192


In [149]:
len(words)

14096

### 4-c-4. word extraction 결과 출력

In [150]:
# 결과 출력

import math

def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy, word_score)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True)[1000:2000]:
    print('%s     (%d, %.3f, %.3f, %.3f)' % (
            word, 
            score.leftside_frequency, 
            score.cohesion_forward,
            score.right_branching_entropy,
            word_score(score)
            )
         )

단어   (빈도수, cohesion, branching entropy, word_score)

마이크론도     (161, 0.275, 3.827, 12.639)
깨지면     (296, 0.294, 3.761, 12.633)
이제     (11713, 0.064, 5.282, 12.621)
성공하면     (262, 0.261, 3.879, 12.609)
개미가     (1075, 0.134, 4.547, 12.602)
냠냠     (260, 0.884, 2.656, 12.589)
백신     (1491, 0.280, 3.805, 12.584)
카카오게임즈는     (154, 0.366, 3.537, 12.580)
삼성은     (816, 0.116, 4.684, 12.580)
뇌피셜     (385, 0.501, 3.222, 12.568)
높은     (2523, 0.278, 3.811, 12.565)
턴어라운드     (155, 0.816, 2.734, 12.562)
점유율이     (226, 0.337, 3.616, 12.542)
과천은     (368, 0.188, 4.198, 12.541)
장기적으로는     (127, 0.360, 3.551, 12.525)
산업이     (354, 0.210, 4.089, 12.516)
받으면     (473, 0.193, 4.170, 12.514)
있을까?     (582, 0.184, 4.218, 12.503)
설마     (1135, 0.149, 4.432, 12.495)
나는     (4625, 0.060, 5.331, 12.479)
회장이     (477, 0.141, 4.479, 12.470)
IT     (1236, 0.334, 3.620, 12.464)
페이스북     (317, 0.466, 3.286, 12.460)
있음?     (1520, 0.128, 4.581, 12.458)
아닌가?     (594, 0.165, 4.324, 12.448)
잡아서     (261, 0.174, 4.269, 1

### 4-c-5. threshold 판단
- 눈으로 직접 살펴보며, 무의미한 결과를 거를 word_score의 threshold를 정한다
- 결과 dict : scored_words

In [151]:
threshold = 1.0
# threshold 를 기준으로 dictionary 생성
scored_words = {word: word_score(score) for word, score in words.items() if word_score(score) >= threshold}

print(scored_words)

{'몰라': 2.393887192976304, '욕하': 1.0169082011530566, '~~': 72.85926999327718, '아님': 2.1544574242910675, '최대': 5.065257385040853, '될지': 2.2742238278090703, '호가': 2.5470687063553736, '썩은': 2.17904451284809, '빚이': 2.0545674943425634, '맘에': 1.159525923823975, '넘어': 1.7779756297625606, '봐도': 50.19493866157249, '드라': 2.4488199651386746, 'ㅈㄴ': 44.828555325501995, '늦게': 11.02216181776753, '등을': 4.496407810131742, '건설': 11.013556396496401, '유럽': 2.786312673655742, '콘솔': 8.16577504631527, 'vs': 70.36003386623821, '형이': 13.87026108373031, '되면': 15.741135506749922, '영상': 2.6824984285848745, '최근': 9.239925770477633, '어케': 3.0795625798769892, '속슬': 1.0275135351482927, '금지': 4.305103490236238, '돼야': 1.8078367376687017, '밴드': 5.508050184765097, '쉬움': 1.7356041660700763, '천억': 5.894281077558707, '행동': 1.0169803732805873, '줏어': 2.733359175366473, '줄이': 2.3707501874868635, 'ㄱㄱ': 46.08873549234032, '니가': 56.75770317382826, '휴젤': 8.571650762659381, '겨우': 41.78707171467094, '감소': 2.1404550403814726, '물량': 9.

In [152]:
# 리스트로 만든다
key_list = list(scored_words.keys())
len(key_list)

7536

### 4-c-6. 숫자 포함된 토큰 삭제

In [153]:
import re

def remove_items_with_numbers(list_a):
    # 정규 표현식을 사용하여 숫자 포함 여부 판단
    return [item for item in list_a if not re.search(r'\d', item)]

# 사용 예시
filtered_list_1 = remove_items_with_numbers(key_list)
len(filtered_list_1) 

7175

### 4-c-7. 조사/접미사 붙은 토큰들 삭제
- "이/가", "을/를", "은/는", "와/과", "도/만/조차", "에/에서/로", "의", "면", "까지/부터", "처럼", "다", "들"
- "." 붙은 것도 삭제
- 이상한 키워드 삭제
- 명사 토큰을 삭제해 버릴 지라도, mecab에서 처리해 주기 때문에 상관없다
- 기준을 엄격히 하여, 이상한 토큰으로 토큰화하지 않는 것이 중요하다. 단어의 원자성을 유지하자.

In [154]:
# 확인해보기
def find_endswith(input_list, endswith):
    result = [item for item in input_list if item.endswith(endswith)]
    return result
    
find_endswith(filtered_list_1, "만원")

['천만원', '십만원', '백만원']

In [198]:
# 함수 : 접미사로 끝나는 원소들을 필터링하여 제거
def remove_items_with_suffixes(list_a, suffixes):
    return [item for item in list_a if not item.endswith(tuple(suffixes))]

suffixes = ["이", "가", "을", "를", "은", "는", "와", "과", "의", "도", "만", "나", "면", "조차", "하고", "이나", "냐", "이라", "고", "때", "거임",
            "듯", "다고", "게", "데", "지", "하냐", "면서", "라고", "요", "까", "동안", "해보", "더니", "임", "됨",
            "에", "에서", "로", "까지", "부터", "처럼", "다", "들", "들아", "야", "니까", "봐", "거", "건", "서", "음", "네", "원", "하", "된", "함",
            "인셍", "ㅋㅋㅋ", "ㅎㅎㅎ", "ㅠㅠㅠ", "ㄷㄷㄷ", "ㅏㅏㅏ", "ㅅㅅㅅ", "ㅇㅇㅇ", "ㄱㄱㄱ",
            "?", "!", ".", ";", ":", "-", "~", "/", "*", "]", ")", "/m" ]

filtered_list_2 = remove_items_with_suffixes(filtered_list_1, suffixes)
len(filtered_list_2)

2752

In [197]:
filtered_list_2

['몰라',
 '아님',
 '최대',
 '넘어',
 '드라',
 'ㅈㄴ',
 '건설',
 '유럽',
 '콘솔',
 'vs',
 '영상',
 '최근',
 '어케',
 '속슬',
 '밴드',
 '쉬움',
 '천억',
 '행동',
 '줏어',
 'ㄱㄱ',
 '휴젤',
 '겨우',
 '감소',
 '물량',
 '채널',
 '통수',
 '첨단',
 '쇼크',
 '요즘',
 '지수',
 '막상',
 '인수',
 '운용',
 '레알',
 '기업',
 '세금',
 '고점',
 '절대',
 '갤주',
 '출시',
 '잠시',
 '명품',
 'ㅆㅂ',
 '욕심',
 '뻘짓',
 '패턴',
 '주식',
 'QQ',
 '섹터',
 '금리',
 '병신',
 'ar',
 '거기',
 '묵히',
 '액분',
 'it',
 '웹툰',
 '찬티',
 '표정',
 '승인',
 '건강',
 '피해',
 'YG',
 '베팅',
 '끌어',
 '횡보',
 '학생',
 '업계',
 '저런',
 '언제',
 '운영',
 '언급',
 'SM',
 'ts',
 '실적',
 '회복',
 '기술',
 '퍄퍄',
 '권리',
 '스트',
 '직업',
 '미래',
 '이제',
 '각각',
 '친구',
 '빙신',
 '꾸준',
 '굿굿',
 '두번',
 'FT',
 '착한',
 '교육',
 '조정',
 '엔터',
 '몇개',
 '치킨',
 '당시',
 'NC',
 'AI',
 '탄생',
 'ㅈㄹ',
 '각종',
 '날아',
 '텔콘',
 '압박',
 '임상',
 '손절',
 '뒤진',
 '미장',
 '급락',
 '유저',
 '증시',
 '세계',
 'ㅇㅇ',
 '서울',
 '컄ㅋ',
 '쪼개',
 '엑바',
 '껄껄',
 '쌍봉',
 '시총',
 'da',
 '드립',
 'ㅇㅈ',
 '휘청',
 '획득',
 '본전',
 '물린',
 '흉기',
 '<<',
 '코인',
 '머리',
 '호구',
 '초기',
 '빌딩',
 '목표',
 '매출',
 '악질',
 '졸업',
 '팀장',
 '괜히',
 '앨범',
 '셀트',

## 4-d. 단어 리스트를 파일로 저장

### 4-e-1. 파일로 저장하기

In [199]:
# 파일로 저장
filename = "4_word_list.txt"
words = filtered_list_2
with open(filename, "w", encoding="utf-8") as file:
    for word in words:
        file.write(word + "\n")

print("파일 저장 완료.")

파일 저장 완료.


### 4-e-2. 불러와서 확인하기

In [201]:
# 파일로부터 문자열 리스트 불러오기
with open(filename, "r", encoding="utf-8") as file:
    loaded_word_list = [line.rstrip('\n') for line in file]

print("불러온 문자열 리스트:")
print(loaded_word_list)

불러온 문자열 리스트:
['몰라', '아님', '최대', '넘어', '드라', 'ㅈㄴ', '건설', '유럽', '콘솔', 'vs', '영상', '최근', '어케', '속슬', '밴드', '쉬움', '천억', '행동', '줏어', 'ㄱㄱ', '휴젤', '겨우', '감소', '물량', '채널', '통수', '첨단', '쇼크', '요즘', '지수', '막상', '인수', '운용', '레알', '기업', '세금', '고점', '절대', '갤주', '출시', '잠시', '명품', 'ㅆㅂ', '욕심', '뻘짓', '패턴', '주식', 'QQ', '섹터', '금리', '병신', 'ar', '거기', '묵히', '액분', 'it', '웹툰', '찬티', '표정', '승인', '건강', '피해', 'YG', '베팅', '끌어', '횡보', '학생', '업계', '저런', '언제', '운영', '언급', 'SM', 'ts', '실적', '회복', '기술', '퍄퍄', '권리', '스트', '직업', '미래', '이제', '각각', '친구', '빙신', '꾸준', '굿굿', '두번', 'FT', '착한', '교육', '조정', '엔터', '몇개', '치킨', '당시', 'NC', 'AI', '탄생', 'ㅈㄹ', '각종', '날아', '텔콘', '압박', '임상', '손절', '뒤진', '미장', '급락', '유저', '증시', '세계', 'ㅇㅇ', '서울', '컄ㅋ', '쪼개', '엑바', '껄껄', '쌍봉', '시총', 'da', '드립', 'ㅇㅈ', '휘청', '획득', '본전', '물린', '흉기', '<<', '코인', '머리', '호구', '초기', '빌딩', '목표', '매출', '악질', '졸업', '팀장', '괜히', '앨범', '셀트', '취급', '랠리', '땅값', '임마', '분할', '과연', '관리', '가능', '온갖', '축소', '줍줍', '떵락', '링크', '념글', '걔넨', '설비', '강한', '흑우', '행사', '격차', '길드', 'c

In [202]:
len(loaded_word_list)

2752